In [1]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [2]:
# Abrir los graficos en el navegador

pio.renderers.default = 'browser'

In [3]:
# Carga de datos

df_ple7= pd.read_csv(r"../data/9-1-2026/data-1768216236753.csv")
df_ple1 = pd.read_csv(r"../data/9-1-2026/data-1768216350513.csv")
# df_data_raw = pd.read_csv(r"../data/data-raw30122025.csv", encoding='latin1')

df_ple1['maquina'] = 'Plegadora 1'
df_ple7['maquina'] = 'Plegadora 7'

# Ordenamiento de los datos en base a su marca temporal
df_ple1 = df_ple1.sort_values('temporal_placa').reset_index(drop=True)  
df_ple7 = df_ple7.sort_values('temporal_placa').reset_index(drop=True)


In [4]:
# Funcion preparar_df
# Esta funcion toma los datos de la plegadora y asigna a cada fila el turno correspondiente 

def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )
    
    # Potencias totales por timestamp
    df['p_activa_total'] = (
        df['potencia_a_r'] +
        df['potencia_a_s'] +
        df['potencia_a_t']
    )

    df['q_reactiva_total'] = (
        df['potencia_r_r'] +
        df['potencia_r_s'] +
        df['potencia_r_t']
    )

    return df


In [5]:
# Ajuste datos de las plegadoras

df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

df_all = df_all.sort_values(['maquina','temporal_placa']).reset_index(drop=True)

# Filtra los datos correspondientes al 10-1-2026
import datetime as dt

dia = dt.date(2026, 1, 9)
df_all = df_all[df_all['temporal_placa'].dt.date == dia]

In [6]:
# Funcion detectar_picos
def detectar_picos(signal, threshold):
    """
    signal: pd.Series
    threshold: float
    Devuelve índices de picos
    """
    return signal[
        (signal > threshold) &
        (signal.shift(1) < signal) &
        (signal.shift(-1) < signal)
    ].index


In [20]:
# Deteccion de picos

import pandas as pd

maquinas = df_all['maquina'].unique()

medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        I_R_media=('corriente_r', 'mean'),
        I_S_media=('corriente_s', 'mean'),
        I_T_media=('corriente_t', 'mean')
    )
)

medias_por_fase['fase_mas_cargada'] = medias_por_fase[
    ['I_R_media', 'I_S_media', 'I_T_media']
].idxmax(axis=1)

fase_col = {
    'I_R_media': 'corriente_r',
    'I_S_media': 'corriente_s',
    'I_T_media': 'corriente_t'
}

fase_label = {
    'I_R_media': 'Fase R',
    'I_S_media': 'Fase S',
    'I_T_media': 'Fase T'
}

# Deteccion
df_peaks_list = []

for maq in maquinas:

    fase_media = medias_por_fase.loc[maq, 'fase_mas_cargada']
    col_corriente = fase_col[fase_media]

    g = (
        df_all[df_all['maquina'] == maq]
        .sort_values('temporal_placa')
        .copy()
    )

    señal = g[col_corriente]

    media = señal.mean()
    UMBRAL = 1.10 * media

    idx_picos = detectar_picos(señal, UMBRAL)

    df_peaks = g.loc[idx_picos, ['temporal_placa']].copy()
    df_peaks['maquina'] = maq
    df_peaks['fase'] = fase_label[fase_media]

    df_peaks_list.append(df_peaks)


# Nuevo dataframe
df_peaks_all = pd.concat(df_peaks_list, ignore_index=True)

df_peaks_all['temporal_placa'] = (
    pd.to_datetime(df_peaks_all['temporal_placa'], errors='coerce')
    .dt.tz_localize(None)
)


In [21]:
# Visualizacion de picos detectados y su marca temporal

df_peaks_all = df_peaks_all.sort_values(
    by=['maquina', 'temporal_placa']
).reset_index(drop=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

#df_peaks_all

In [22]:
# Analisis de ciclos
import pandas as pd

# Parámetro clave
SILENCIO_MAX = pd.Timedelta(minutes=2)

rows = []

for maq, g in df_peaks_all.groupby('maquina'):

    g = g.sort_values('temporal_placa').reset_index(drop=True)

    # diferencia entre picos consecutivos
    g['dt'] = g['temporal_placa'].diff()

    # nuevo ciclo si:
    # - es el primer pico
    # - o el silencio supera 1 minuto
    g['nuevo_ciclo'] = (g['dt'].isna()) | (g['dt'] > SILENCIO_MAX)

    # id de ciclo incremental
    g['ciclo_id'] = g['nuevo_ciclo'].cumsum()

    # agrupar por ciclo
    for ciclo_id, c in g.groupby('ciclo_id'):

        fecha_inicio = c['temporal_placa'].iloc[0]
        fecha_fin    = c['temporal_placa'].iloc[-1]

        golpes_ciclo = max(len(c) - 1, 0)

        rows.append({
            'maquina': maq,
            'ciclo_id': ciclo_id,
            'golpes_ciclo': golpes_ciclo,
            'fecha_inicio': fecha_inicio,
            'fecha_fin': fecha_fin,
            'duracion': fecha_fin - fecha_inicio
        })

df_ciclos = pd.DataFrame(rows)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df_ciclos

,maquina,ciclo_id,golpes_ciclo,fecha_inicio,fecha_fin,duracion
0,Plegadora 1,1,3,2026-01-09 16:57:16,2026-01-09 16:58:04,0 days 00:00:48
1,Plegadora 1,2,2,2026-01-09 17:04:56,2026-01-09 17:05:41,0 days 00:00:45
2,Plegadora 1,3,2,2026-01-09 17:11:30,2026-01-09 17:11:57,0 days 00:00:27
3,Plegadora 1,4,10,2026-01-09 17:15:37,2026-01-09 17:20:40,0 days 00:05:03
4,Plegadora 1,5,1,2026-01-09 18:07:26,2026-01-09 18:07:38,0 days 00:00:12
5,Plegadora 1,6,1,2026-01-09 18:09:52,2026-01-09 18:10:05,0 days 00:00:13
6,Plegadora 1,7,4,2026-01-09 18:14:02,2026-01-09 18:15:20,0 days 00:01:18
7,Plegadora 1,8,3,2026-01-09 18:17:47,2026-01-09 18:18:37,0 days 00:00:50
8,Plegadora 1,9,2,2026-01-09 18:20:44,2026-01-09 18:21:12,0 days 00:00:28
9,Plegadora 1,10,2,2026-01-09 18:29:01,2026-01-09 18:29:26,0 days 00:00:25
